# DETOXIFICATION

In [1]:
%pip install -q torch
%pip install -q transformers[torch]
%pip install -q googletrans==3.1.0a0
%pip install -q fastparquet
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Lectura de datos

Vamos a leer el dataset de entrenamiento. Este dataset esta formado por 400 frases tóxicas por idioma en 9 idiomas diferentes y sus versiones no tóxicas. Formaremos un dataset conjunto que contenga para cada frase, su idioma de origen, la frase tóxica y la frase de-toxificada.

In [16]:
splits = {'en': 'data/en-00000-of-00001.parquet', 'ru': 'data/ru-00000-of-00001.parquet', 'uk': 'data/uk-00000-of-00001.parquet', 'de': 'data/de-00000-of-00001.parquet', 'es': 'data/es-00000-of-00001.parquet', 'am': 'data/am-00000-of-00001.parquet', 'zh': 'data/zh-00000-of-00001.parquet', 'ar': 'data/ar-00000-of-00001.parquet', 'hi': 'data/hi-00000-of-00001.parquet'}
df_en = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["en"])
df_uk = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["uk"])
df_ru = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ru"])
df_de = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["de"])
df_es = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["es"])
df_am = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["am"])
df_zh = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["zh"])
df_ar = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ar"])
df_hi = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["hi"])

df_hf = pd.concat([df_en, df_uk, df_ru, df_de, df_es, df_am, df_zh, df_ar, df_hi], keys=['en','uk', 'ru', 'de', 'es', 'am', 'zh-cn', 'ar', 'hi']).reset_index(level=[0])
df_hf.sample(3)

,level_0,toxic_sentence,neutral_sentence
126,de,Der Hank geht mir dermaßen auf die Eier.,Der Hank geht mir dermaßen auf die Nerven.
155,de,Bei einer psycho Sekte von freiwillig zu sprec...,Bei einer Sekte von freiwillig zu sprechen ist...
115,de,Kotz deine Propaganda wo anders aus. Hier bist...,Werde deine Propaganda wo anders los. Hier bis...


El siguiente dataset es de github, contiene 500 ejemplos en inglés

In [18]:
df_gh = pd.read_csv("https://raw.githubusercontent.com/panlingua/multilingual_text_detoxification_datasets/refs/heads/main/refined-english/en_parallel_detoxification.test.csv")
df_gh.rename(columns={"toxic_comment":"toxic_sentence", "civil_comment":"neutral_sentence"}, inplace=True)
df_gh.sample(3)

,ID,toxic_sentence,neutral_sentence
293,1266,it s hard to keep track of which dicknoses dow...,It is difficult to keep track of people that b...
222,722,gt obama is a lying sack of shit .,gt obama is unreliable person.
452,1070,"i mean , how much does that suck ?",i mean how much does that hurts?


dataset que nos da la tarea

In [23]:
df_tarea=pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")
df_tarea.rename(columns={"en_toxic_comment":"toxic_sentence", "en_civil_comment":"neutral_sentence"}, inplace=True)
df_tarea.sample(3)

,toxic_sentence,en_neutral_comment
18600,don 't really want to fill all my info in and ...,Don't really want to fill all my info in and g...
17955,your a liberal fuck up .,You are a liberal
17820,i don 't even know what to say about this shit .,I dont even know what to say about this


📝 Finalmente, se define el **dataset final** que utilizará los dataframes anteriormente implementados.

In [ ]:
# df_total = pd.concat([df_hf, df_gh])
df_total = df_tarea

In [20]:
print(df_total.shape)
df_total.sample(3)

(4100, 4)


,level_0,toxic_sentence,neutral_sentence,ID
346,es,Luego si empiezan a aparecer los cadáveres de ...,Luego si empiezan a aparecer los cadáveres alg...,NaN
199,ar,لعنة الله ع طبقتنا السياسية الفاسدة والفاشلة ه...,حسبي الله ع طبقتنا السياسية الفاسدة والفاشلة ه...,NaN
333,en,who the hell do you feminists think you are an...,Who do these feminists think they are anyway?,NaN


### Train-test split

In [ ]:
# train, test = train_test_split(df_total, train_size=0.8, random_state=33)

X_train = list(df_total['toxic_sentence'])
y_train = list(df_total['neutral_sentence'])

X_test = list(df_hf['toxic_sentence'])
y_test = list(df_hf['neutral_sentence'])
len(X_test)

720

## Pipeline de detoxificación

Nuestro modelo de detoxificación tendrá un pipeline que consistirá de los siguientes pasos:

1. **Traducción**: Traduciremos las frases del idioma original al inglés
2. **Detoxificación**: El modelo detoxificará la frase en inglés
3. **Retro-traducción**: La frase detoxificada será traducida del inglés al idioma original

### Traducción y retrotraducción

Usaremos la libreria de google translate para crear un modelo que traduzca un texto de un lenguaje a otro, pasados por argumentos.

In [7]:
from googletrans import Translator

translator = Translator()

In [8]:
def translate(text, source_lang, dest_lang):
    if source_lang == dest_lang:
        return text

    sol = translator.translate(text , src = source_lang, dest=dest_lang).text
    return sol

In [9]:
test_trad = df_total.sample(5).to_numpy()

for sentence in test_trad:
    print(f"Original sentence ({sentence[0]}): {sentence[1]}")
    translation = translate(sentence[1], sentence[0], "en")
    print(f"Transalation: {translation}")
    print("-"*50)

Original sentence (de): Auch ein Merkel Vasall der das sagen muss was diese Volksverräterin vorgibt!!! Lügenpresse!!!
Transalation: Also a Merkel vassal who has to say what this traitor says!!! Lying press!!!
--------------------------------------------------
Original sentence (de): Kotz deine Propaganda wo anders aus. Hier bist du falsch.
Transalation: Puke your propaganda somewhere else. You are wrong here.
--------------------------------------------------
Original sentence (ar): بيكيه ماعليه ياعمري صح انك شوي غبي بس عادي اهم شيء اشوفك ياعمري
Transalation: What's wrong with you, my love? It's true that you're a little stupid, but that's okay. The most important thing is to see you, my love
--------------------------------------------------
Original sentence (en): this is a special needs child with autism , not some angry little shit with no self control .
Transalation: this is a special needs child with autism , not some angry little shit with no self control .
---------------------

## Detoxificación

Ahora haremos un finetunning de un modelo de Huggingface para detoxificar el código.

In [10]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

num_training_layers = 2 # El número de capas que se entrenarán

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas n capas del decoder
for param in model.decoder.block[-num_training_layers:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ESTADÍSTICAS DEL ENTRENAMIENTO:")
print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


ESTADÍSTICAS DEL ENTRENAMIENTO:
Total de parámetros: 76,961,152
Parámetros entrenables: 22,744,064
Parámetros congelados: 54,217,088


#### Creación de los dataset

In [11]:
class DetoxificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        # Reemplazar pad_token_id con -100 para ignorar tokens de relleno en la pérdida
        item["labels"][item["labels"] == tokenizer.pad_token_id] = -100

        return item

In [12]:
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
y_train_tokenized = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

X_test_tokenized = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")
y_test_tokenized = tokenizer(y_test, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

print("Tamaño del dataset de entrenamiento:", len(train_dataset))
print("Ejemplo del dataset:", train_dataset[0])


# training_args = TrainingArguments(
#     output_dir='./checkpoints',
#     eval_strategy="no",
#     per_device_train_batch_size=128,
#     learning_rate=1e-4,
#     num_train_epochs=5,
#     report_to=["none"],
#     #fp16=True, # acelerar entrenaminento
# )


Tamaño del dataset de entrenamiento: 2880
Ejemplo del dataset: {'input_ids': tensor([ 3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,  3,  2,
         3,  2,  3,  2,  3,  2,  3,  2,  3,  2, 55,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 

## Entrenamiento

In [13]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm import tqdm

def train_t5(model, train_dataset, val_dataset, tokenizer, epochs=5, batch_size=8, lr=5e-5, device="cuda"):
    # Enviar modelo a dispositivo
    model.to(device)
    model.train()

    # DataLoaders para entrenamiento y validación
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizador y scheduler
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = epochs * len(train_loader)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    # Definir función de pérdida
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)  # Ignora los tokens -100

    # Loop de entrenamiento
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss = 0

        for batch in tqdm(train_loader, desc="Training"):
            # Preparar inputs y labels
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Adelante y cálculo de pérdida
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

            # Retropropagación y actualización de parámetros
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        print(f"Average training loss for epoch {epoch + 1}: {avg_loss}")

        # Validación después de cada época
        evaluate_t5(model, val_loader, loss_fn, device)
        model.save_pretrained("detox_model")

    return model

def evaluate_t5(model, val_loader, loss_fn, device):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Adelante y cálculo de pérdida
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation loss: {avg_val_loss}")
    model.train()


In [ ]:
trained_model = train_t5(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    tokenizer=tokenizer,
    epochs=10,
    batch_size=16,
    lr=1e-5,
    device=device
)


C:\Users\mgonz\anaconda3\envs\python-3-10\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10


Training: 100%|██████████████████████████████████████████████████████████████████████| 720/720 [10:09<00:00,  1.18it/s]


Average training loss for epoch 1: 1.2164266583199301


Validation: 100%|████████████████████████████████████████████████████████████████████| 180/180 [01:14<00:00,  2.40it/s]


...

Average training loss for epoch 3: 1.1136038463769689


Validation: 100%|████████████████████████████████████████████████████████████████████| 180/180 [01:15<00:00,  2.37it/s]


Validation loss: 0.8579542361199856
Epoch 4/10


Training:  35%|████████████████████████▌                                             | 252/720 [03:30<06:31,  1.20it/s]


KeyboardInterrupt: 

In [ ]:
#guardar
trained_model.save_pretrained("detox_model")

In [17]:
#cargar y probar
loaded_model = T5ForConditionalGeneration.from_pretrained("detox_model")
loaded_model.to(device)


frase="Russia is a country of fuckers. everyone offers something, but no one does anything"

loaded_model.eval()
input_ids = tokenizer(frase, return_tensors="pt").input_ids.to(device)
outputs = loaded_model.generate(input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)



'.'